In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import multiprocessing
import math
import string
import sdeint
import random
import statistics
import scipy.signal
import scipy.stats as st

from scipy.stats import lognorm, expon, rv_continuous, kurtosis, ttest_ind
from scipy import stats,optimize
from scipy.integrate import odeint
from scipy.optimize import curve_fit
from scipy.special import gamma, digamma

from joblib import Parallel, delayed 
from matplotlib import gridspec
from statsmodels.tsa.seasonal import seasonal_decompose
from itertools import accumulate
from collections import Counter
from datetime import datetime,timedelta
from pandas.plotting import register_matplotlib_converters

import os
import elevation
#from PyEMD import EMD
register_matplotlib_converters()

In [ ]:
#read
TBB_data = pd.read_csv('Dataframes_new_new/TBB_data.csv')
TKB_data = pd.read_csv('Dataframes_new_new/TKB_data.csv')
TChP_data = pd.read_csv('Dataframes_new_new/TChP_data.csv')
TH_data = pd.read_csv('Dataframes_new_new/TH_data.csv')
TPur_data = pd.read_csv('Dataframes_new_new/TPur_data.csv')
TPut_data = pd.read_csv('Dataframes_new_new/TPut_data.csv')
TCaP_data = pd.read_csv('Dataframes_new_new/TCaP_data.csv')
TBGP_data = pd.read_csv('Dataframes_new_new/TBGP_data.csv')
TEB_data = pd.read_csv('Dataframes_new_new/TEB_data.csv')

In [ ]:
dataframe_names = ['TBB_data', 'TKB_data', 'TChP_data', 'TH_data', 'TPur_data', 'TPut_data', 'TCaP_data', 'TBGP_data', 'TEB_data']
day = 24*60*60 #seconds
year = (365.2425)*day

for name in dataframe_names:
    # Read the csv file
    df = pd.read_csv('Dataframes_new_new/'+name + '.csv')
    df = pd.read_csv('Dataframes_new_new/'+name + '.csv')
    df.pop('DATE')
    df.pop('TIME')
    df.pop('Hour of Day')
    df.pop('Month')
    df.pop('Day of Week')
    df.pop('dist_to_sea (km)')
    df.pop('DOO (percent saturation) Class')
    df = df.dropna()
 
    date_time = df['DATE TIME']
    date_time = pd.to_datetime(date_time, format='%Y-%m-%d %H:%M:%S')
    timestamp_s = date_time.map(pd.Timestamp.timestamp)
    
    #returns component of each datetime to range from -0.5 to 0.5
    df['Hour of Day'] = date_time.dt.hour/23-0.5
    df['Day of Week'] = date_time.dt.dayofweek/6-0.5
    df['Month of Year'] = (date_time.dt.month-1)/11-0.5

    df['Half Day sin'] = np.sin(timestamp_s * (2 * np.pi / (day/2)))
    df['Half Day cos'] = np.cos(timestamp_s * (2 * np.pi / (day/2)))

    df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))  
    
    if name in ['TBB_data', 'TKB_data', 'TH_data','TChP_data']:     
        df.pop('OB_END_TIME')    
    #include TPut
    if name in ['TBGP_data', 'TEB_data']:     
        df.pop('OB_DATE')   
        
    if name in ['TBB_data', 'TKB_data', 'TH_data', 'TPut_data', 'TCaP_data', 'TBGP_data', 'TEB_data']:     
        #Salinity same as EC
        df.pop('SALINITY')

    df.to_csv('Dataframes_new/'+name + '.csv', index=False)

In [ ]:
# check if a string is NAN   
def isNaN(string):
    return string != string
#path
path_sites = 'E:\\PHD\Water\EA'

#TEB_data['DATE TIME'] = TEB_data['DATE TIME'].apply(pd.to_datetime)
#new_col = TEB_data['DATE TIME'].dt.month 
#TEB_data.insert(loc=11, column='Month', value=new_col)

Due to a date change in path_sites+ ..csv, the current doesn't correctly reflect the datasets we used.

In [ ]:
#Read Thames Barrier Gardens Pier data
TBGP_data=pd.read_csv(path_sites+'\BARIERA.csv')
#remove zeros
TBGP_data[['TEMP','COND','PH','AMMONIUM','TURBIDITY','DOO','DOO-MGL','BATTERY','SALINTY']]=\
TBGP_data[['TEMP','COND','PH','AMMONIUM','TURBIDITY','DOO','DOO-MGL','BATTERY','SALINTY']].replace(0,np.nan)
#remove negatives
TBGP_data[TBGP_data[['COND','PH','AMMONIUM','DOO','DOO-MGL','BATTERY','SALINTY']]<0] = np.nan
#filter corrupted data
TBGP_data['AMMONIUM'][0:20950] = np.nan
TBGP_data['AMMONIUM'][32353:35327] = np.nan
TBGP_data['AMMONIUM'][146103:] = np.nan

#Read Thames Brentford Badge data
TBB_data=pd.read_csv(path_sites+'\BREPON.csv')
TBB_data[['TEMP','COND','PH','AMMONIUM','TURBIDITY','DOO','DOO-MGL','BATTERY','SALINITY']]=\
TBB_data[['TEMP','COND','PH','AMMONIUM','TURBIDITY','DOO','DOO-MGL','BATTERY','SALINITY']].replace(0,np.nan)
TBB_data[TBB_data[['COND','PH','AMMONIUM','DOO','DOO-MGL','BATTERY','SALINITY']]<0] = np.nan
#Filter out EC<10 due to sudden dropout
TBB_data['COND'][TBB_data['COND']<10] = np.nan

#Read Thames Cadogan Pier data
TCaP_data=pd.read_csv(path_sites+'\CADOG2.csv')
TCaP_data[['TEMP','COND','PH','AMMONIUM','TURBIDITY','DOO','DOO-MGL','BATTERY','SALINITY']]=\
TCaP_data[['TEMP','COND','PH','AMMONIUM','TURBIDITY','DOO','DOO-MGL','BATTERY','SALINITY']].replace(0,np.nan)
TCaP_data[TCaP_data[['COND','PH','AMMONIUM','DOO','DOO-MGL','BATTERY','SALINITY']]<0] = np.nan

#Read Thames Chiswick Pier data
TChP_data=pd.read_csv(path_sites+'\GPRSD8A.csv')
TChP_data[['TEMP','COND','PH','AMMONIUM','TURBIDITY','DOO','DOO-MGL','BATTERY']]=\
TChP_data[['TEMP','COND','PH','AMMONIUM','TURBIDITY','DOO','DOO-MGL','BATTERY']].replace(0,np.nan)
TChP_data[TChP_data[['COND','PH','AMMONIUM','DOO','DOO-MGL','BATTERY']]<0] = np.nan
#Filter out faulty data
TChP_data['DOO'][91647:94328]=np.nan 
TChP_data['DOO-MGL'][91647:94328]=np.nan 

#Read Thames Erith Badge data
TEB_data=pd.read_csv(path_sites+'\ERITH1.csv')
TEB_data[['TEMP','COND','PH','TURBIDITY','DOO','DOO-MGL','BATTERY','SALINITY']]=\
TEB_data[['TEMP','COND','PH','TURBIDITY','DOO','DOO-MGL','BATTERY','SALINITY']].replace(0,np.nan)
TEB_data[TEB_data[['COND','PH','DOO','DOO-MGL','BATTERY','SALINITY']]<0] = np.nan
#Some EC measurements in ms unit, change to us
TEB_data['COND'][0:30860] = TEB_data['COND'][0:30860]*1000
TEB_data['COND'][33844:89695] = TEB_data['COND'][33844:89695]*1000
TEB_data['COND'][89992:] = TEB_data['COND'][89992:]*1000
#remove hourly measurement
TEB_data = TEB_data[3371:]
#remove doo-mgl sudden fluctuations
TEB_data['DOO-MGL'][21846:21868] = np.nan
TEB_data = TEB_data[TEB_data['DOO-MGL']<25]

#Read Thames Hammersmith data
TH_data=pd.read_csv(path_sites+'\HAMME2.csv')
TH_data[['TEMP','COND','PH','AMMONIUM','TURBIDITY','DOO','DOO-MGL','BATTERY','SALINITY']]=\
TH_data[['TEMP','COND','PH','AMMONIUM','TURBIDITY','DOO','DOO-MGL','BATTERY','SALINITY']].replace(0,np.nan)
TH_data[TH_data[['COND','PH','AMMONIUM','DOO','DOO-MGL','BATTERY','SALINITY']]<0] = np.nan
#data failure before sonde change
TH_data['AMMONIUM'][93935:99845] =np.nan
#Filter EC huge spikes:
TH_data['COND'][117335:119522] =np.nan
TH_data['COND'][119723:120966] =np.nan
TH_data['COND'][121014:121150] =np.nan
TH_data['COND'][121179:121253] =np.nan
TH_data['COND'][121282:121287] =np.nan
TH_data['COND'][121326:121412] =np.nan
TH_data['COND'][121517:121752] =np.nan

#Read Thames Kew Badge data
TKB_data=pd.read_csv(path_sites+'\KEWPON.csv')
TKB_data[['TEMP','COND','PH','AMMONIUM','TURBIDITY','DOO','DOO-MGL','BATTERY','SALINITY']]=\
TKB_data[['TEMP','COND','PH','AMMONIUM','TURBIDITY','DOO','DOO-MGL','BATTERY','SALINITY']].replace(0,np.nan)
TKB_data[TKB_data[['COND','PH','AMMONIUM','DOO','DOO-MGL','BATTERY','SALINITY']]<0] = np.nan
#Ammonium 9999, hence remove        
TKB_data['AMMONIUM'][76514:76515] = np.nan
#Filter faulty data
TKB_data['AMMONIUM'][76536:76865] = np.nan
#AMMONIUM sonde probe fault:
TKB_data['AMMONIUM'][164299:166555] = np.nan
#Erith sonde used:
TKB_data['AMMONIUM'][50637:51688] =np.nan      


#Read Thames Purfleet data
TPur_data=pd.read_csv(path_sites+'\E03036A.csv')
TPur_data[['TEMP','COND','PH','TURBIDITY','DOO-pcent','DOO-MGL','BATTERY']]=\
TPur_data[['TEMP','COND','PH','TURBIDITY','DOO-pcent','DOO-MGL','BATTERY']].replace(0,np.nan)
TPur_data[TPur_data[['COND','PH','DOO-pcent','DOO-MGL','BATTERY']]<0] = np.nan
#Filter faulty EC
TPur_data['COND'][15487:15489] = np.nan
#change unit ms to us
TPur_data['COND'] = TPur_data['COND']*1000
#remove spiky /sudden drop of doo-mgl
TPur_data['DOO-MGL'][106900:107380]=np.nan
TPur_data = TPur_data[TPur_data['DOO-MGL']<25]

#Read Thames Putney data
TPut_data=pd.read_csv(path_sites+'\PUTNEY.csv')
TPut_data[['TEMP','COND','PH','AMMONIUM','TURBIDITY','DOO-pcent','DOO-MGL','BATTERY','SALINITY']]=\
TPut_data[['TEMP','COND','PH','AMMONIUM','TURBIDITY','DOO-pcent','DOO-MGL','BATTERY','SALINITY']].replace(0,np.nan)
TPut_data[TPut_data[['COND','PH','AMMONIUM','DOO-pcent','DOO-MGL','BATTERY','SALINITY']]<0] = np.nan
#EC faulty due to sonde change        
TPut_data['COND'][154307:154480] =np.nan
#Ammonium faulty data   
TPut_data['AMMONIUM'][154352:154382] =np.nan
TPut_data['AMMONIUM'][154479:154480] =np.nan

TBB_data['dist_to_sea (km)']=61.39
TKB_data['dist_to_sea (km)']=59.96
TChP_data['dist_to_sea (km)']=57.78
TH_data['dist_to_sea (km)']=56.64
TPut_data['dist_to_sea (km)']= 55.43
TCaP_data['dist_to_sea (km)']= 51.91
TBGP_data['dist_to_sea (km)']= 37.5
TEB_data['dist_to_sea (km)']= 28.77
TPur_data['dist_to_sea (km)']=23.26



In [ ]:
TEB_data['COND'] = np.where(TEB_data['COND']<40000, TEB_data['COND'], np.nan)
TBGP_data['COND'] = np.where(TBGP_data['COND']<25000, TBGP_data['COND'], np.nan)
TPur_data['COND'] = np.where(TPur_data['COND']<40000, TPur_data['COND'], np.nan)
TH_data['COND'] = np.where(TH_data['COND']<5000, TH_data['COND'], np.nan)
TChP_data['COND'] = np.where(TChP_data['COND']<5000, TChP_data['COND'], np.nan)

In [ ]:
TEB_data = TEB_data[TEB_data['COND']<40000]
TBGP_data = TBGP_data[TBGP_data['COND']<25000]
TPur_data = TPur_data[TPur_data['COND']<40000]
TH_data = TH_data[TH_data['COND']<5000]
TChP_data = TChP_data[TChP_data['COND']<5000]

In [ ]:
dataframe_names = ['TBB_data', 'TKB_data', 'TChP_data', 'TH_data', 'TPur_data', 'TPut_data', 'TCaP_data', 'TBGP_data', 'TEB_data']

for name in dataframe_names:
    # Read the csv file
    df = pd.read_csv('Dataframes/'+name + '.csv')
    df['DATE TIME'] = df['DATE'] + ' ' + df['TIME']
    
    if name in ['TBGP_data', 'TPut_data']:
        # Convert using the '%Y-%m-%d %H:%M:%S' format for these two dataframes
        df['DATE TIME'] = df['DATE TIME'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S'))
    else:
        # Convert using the '%d/%m/%Y %H:%M:%S' format for the rest of the dataframes
        df['DATE TIME'] = df['DATE TIME'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S'))

    df.to_csv('Dataframes/'+name + '.csv', index=False)

In [ ]:
#Remove doo data affected by the injection of oxygen in August 2022
dataframe_names = ['TBB_data', 'TKB_data', 'TChP_data', 'TH_data', 'TPut_data', 'TCaP_data', 'TBGP_data', 'TEB_data']

# Set start and end times
start_time = pd.to_datetime('2022-08-17 12:00:00')
end_time = pd.to_datetime('2022-09-01 00:00:00')

for name in dataframe_names:
    # Read the csv file
    df = pd.read_csv('Dataframes/'+name + '.csv')

    # Ensure 'DATE TIME' is in the datetime format
    df['DATE TIME'] = pd.to_datetime(df['DATE TIME'])
    
    # Create a mask for the time period of interest
    mask = (df['DATE TIME'] >= start_time) & (df['DATE TIME'] <= end_time)

    # Set 'DOO-MGL' to np.nan for the time period
    df.loc[mask, 'DOO-MGL'] = np.nan

    # Save the modified dataframe back to a new csv file
    df.to_csv('Dataframes/'+name + '.csv', index=False)

In [ ]:
TBB_data.to_csv('Dataframes/TBB_data.csv', index=False)
TKB_data.to_csv('Dataframes/TKB_data.csv', index=False)
TChP_data.to_csv('Dataframes/TChP_data.csv', index=False)
TH_data.to_csv('Dataframes/TH_data.csv', index=False)
TPur_data.to_csv('Dataframes/TPur_data.csv', index=False)
TPut_data.to_csv('Dataframes/TPut_data.csv', index=False)
TCaP_data.to_csv('Dataframes/TCaP_data.csv', index=False)
TBGP_data.to_csv('Dataframes/TBGP_data.csv', index=False)
TEB_data.to_csv('Dataframes/TEB_data.csv', index=False)